In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
melt_file = 'IN_fullbasin_meltcontributionstytype_500m.txt'
precip_file = 'IN_v01_fullbasinmasks.MERRA_rainfall_km3.annual_mean.by_elev.txt'

melt_df = pd.read_csv(melt_file, sep='\s+')
print(melt_df)

prec_df = pd.read_csv(precip_file, sep='\s+')
print(prec_df)




In [ ]:
melt_cum = melt_df.sort_index(ascending=False).cumsum()
prec_cum = prec_df.sort_index(ascending=False).cumsum()

print(melt_cum)
print(prec_cum)

In [ ]:
all_contribs = pd.concat((melt_cum, prec_cum), axis=1)

print(all_contribs)

In [ ]:
# Calculate contributions as fraction of total.  Drop all rows with all zeros first.

all_contribs = all_contribs[(all_contribs.T != 0).any()]        # drop zero rows

sums = all_contribs.sum(axis=1)
print("SUMS\n", sums)
all_contribs_fraction = all_contribs.divide(sums.values, axis='index')
all_contribs_fraction.index.name = 'Elevation (m)'

print("FRACTIONS")
print(all_contribs_fraction)

In [ ]:
all_contribs_fraction.sum(axis=1)

In [ ]:
ax = all_contribs_fraction.plot(title='Cumulative streamflow contribution with distance down-river')
ax.set_ylabel('Fractional stream contribution')

In [ ]:
ax = all_contribs.plot(title='Cumulative streamflow contribution with distance down-river')
ax.set_ylabel(r'Stream contribution (km$^3$)')

The above plots look reasonable except that the "rain" component seems too high.  I think these numbers include all precipitation, including both rain and snow.  Therefore, I need to remove the snow somehow.

One way would be to linearly interpolate between a snow fraction at some high elevation and zero snow at a lower elevation.

In [ ]:
high_elev = 7500
snow_frac_high = 0.7
low_elev = 3000
snow_frac_low = 0.1

elev = np.array(prec_df.index)

snow_fractions = (snow_frac_high - snow_frac_low)*(elev - low_elev)/(high_elev - low_elev) + snow_frac_low
snow_fractions[snow_fractions < 0.0] = 0.0
print("SNOW FRACTIONS\n", snow_fractions)

prec_df_rainonly = prec_df
prec_df_rainonly['precip'] = prec_df_rainonly['precip'] * (1.0 - snow_fractions)

prec_cum = prec_df_rainonly.sort_index(ascending=False).cumsum()

all_contribs_rainonly = pd.concat((melt_cum, prec_cum), axis=1)

print("ALL_CONTRIBS_RAINONLY\n", all_contribs_rainonly)

In [ ]:
ax = all_contribs_rainonly.plot(title='Cumulative streamflow contribution with distance down-river')
ax.set_ylabel(r'Stream contribution (km$^3$)')

In [ ]:
all_contribs_rainonly = all_contribs_rainonly[(all_contribs_rainonly.T != 0).any()]        # drop zero rows

sums = all_contribs_rainonly.sum(axis=1)
print("SUMS\n", sums)
all_contribs_rainonly_fraction = all_contribs.divide(sums.values, axis='index')
all_contribs_rainonly_fraction.index.name = 'Elevation (m)'

print("FRACTIONS")
print(all_contribs_rainonly_fraction)

In [ ]:
all_contribs_rainonly_fraction = all_contribs_rainonly.divide(sums.values, axis='index')

In [ ]:
ax = all_contribs_rainonly_fraction.plot(title='Cumulative streamflow contribution with distance down-river')
ax.set_ylabel('Fractional stream contribution')